In [117]:
import numpy as np

In [118]:
def perm_function(x, beta=10):
    x1, x2 = x
    S1 = (1 + beta) * (x1 - 1) + (2 + beta) * (x2 - 0.5)
    S2 = (1 + beta) * (x1**2 - 1) + (2 + beta) * (x2**2 - 0.25)
    return S1**2 + S2**2

In [119]:
# Gradient
def perm_function_gradient(x, beta=10):
    x1, x2 = x
    S1 = (1 + beta) * (x1 - 1) + (2 + beta) * (x2 - 0.5)
    S2 = (1 + beta) * (x1**2 - 1) + (2 + beta) * (x2**2 - 0.25)
    grad_x1 = 2 * (1 + beta) * S1 + 4 * (1 + beta) * x1 * S2
    grad_x2 = 2 * (2 + beta) * S1 + 4 * (2 + beta) * x2 * S2
    return np.array([grad_x1, grad_x2])

In [120]:
# Hessian
def perm_function_hessian(x, beta=10):
    x1, x2 = x
    S1 = (1 + beta) * (x1 - 1) + (2 + beta) * (x2 - 0.5)
    S2 = (1 + beta) * (x1**2 - 1) + (2 + beta) * (x2**2 - 0.25)
    H11 = 2 * (1 + beta)**2 + 8 * (1 + beta)**2 * x1**2 + 4 * (1 + beta) * S2
    H22 = 2 * (2 + beta)**2 + 8 * (2 + beta)**2 * x2**2 + 4 * (2 + beta) * S2
    H12 = 2 * (1 + beta) * (2 + beta) + 8 * (1 + beta) * (2 + beta) * x1 * x2
    return np.array([[H11, H12], [H12, H22]])

In [121]:
def constraint(x):
    x1, x2 = x
    return x2 - (x1**2 - 1)

In [122]:
def constraint_jacobian(x):
    x1, x2 = x
    return np.array([-2 * x1, 1])

In [123]:
def newton_solver(x0, tol=1e-6, max_iter=50, beta=10):
    x = np.array(x0)
    for i in range(max_iter):
        grad = perm_function_gradient(x, beta)
        hess = perm_function_hessian(x, beta)
        g = constraint(x)
        g_jac = constraint_jacobian(x)
        KKT_matrix = np.block([
            [hess, g_jac[:, None]],
            [g_jac[None, :], np.zeros((1, 1))]
        ])
        KKT_rhs = -np.append(grad, g)
        try:
            step = np.linalg.solve(KKT_matrix, KKT_rhs)
        except np.linalg.LinAlgError:
            raise ValueError("Hessian is ill-conditioned.")
        delta_x = step[:-1]
        x += delta_x
        if np.linalg.norm(delta_x) < tol:
            print(f"Converged in {i + 1} iterations.")
            return x, perm_function(x, beta)

    raise ValueError("Did not converge.")

In [124]:
# Initial guess
x0 = [0.5, 0.5]

try:
    solution, objective_value = newton_solver(x0)
    print("Optimal solution:", solution)
    print("Objective function optimum:", objective_value)
except ValueError as e:
    print("Error:", e)

Converged in 7 iterations.
Optimal solution: [1.13282148 0.2832845 ]
Objective function optimum: 2.463096899086052
